# This notebook shows an example of leaning layered architecture of reconfigurable linear optical interferometer

In [1]:
from iloptics.layered import Layered
from iloptics.benchmarks import df95, dt_max

### Create a dummy instance of interferometer and get the measurment protocol to learn the mixing layers

In [2]:
dim = 5  # Number of input/output modes
max_columns = 1  # Maximum columns of mixing layer to learn at once
lt = Layered.dummy(dim)
proto = lt.learn_proto(max_columns)

### Simulate the experiment

In [3]:
# Create an instance of true interferometer
lt_true = Layered.generate(
    dim=dim,
    control_noise=1e-4,
    control_cross_talk=0.1,
    hadamard_like=True,
    hadamard_error=0.1,
    non_uniform_losses=0.,
    noise_tomo=1e-4
)

# Simulate the measurements according to protocol
data = []
for p in proto:
    lt_true.reset()
    if p.controls is not None:
        lt_true.control_layer(p.meta['phase_layer_idx'], p.controls)
    data.append(lt_true.tomo())

### Do the learning using the unitary constraint and print benchmark values

In [4]:
lt.learn(proto, data, uni=True)
print('Prediction error:', df95(lt, lt_true, 100))
print('Transmission coefficients reconstruction error:', dt_max(lt, lt_true))

Prediction error: 4.7044402802043346e-05
Transmission coefficients reconstruction error: 0.00012302994788498944
